In [14]:
# First import a bunch of useful stuff

import os, subprocess
import json
import uproot3
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pickle

from plotter import *

In [15]:
# This gives the integrated luminosity (the amount of data collected) for each year
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

In [16]:
# This dictionary gives the cross section (expected rate) of each simulated process
# Number of events of process P = integrated luminosity * cross section of process P
with open('xsec.json') as f:
  xs = json.load(f)
print(xs)

{'QCD_HT500to700_TuneCUETP8M1_13TeV-madgraphMLM-pythia8': 32200, 'QCD_HT700to1000_TuneCUETP8M1_13TeV-madgraphMLM-pythia8': 6839, 'QCD_HT1000to1500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8': 1207, 'QCD_HT1500to2000_TuneCUETP8M1_13TeV-madgraphMLM-pythia8': 120.1, 'QCD_HT2000toInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8': 25.24, 'QCD_HT300to500_TuneCP5_13TeV-madgraph-pythia8': 322600, 'QCD_HT500to700_TuneCP5_13TeV-madgraph-pythia8': 29980, 'QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8': 6334, 'QCD_HT1000to1500_TuneCP5_13TeV-madgraph-pythia8': 1088, 'QCD_HT1500to2000_TuneCP5_13TeV-madgraph-pythia8': 99.11, 'QCD_HT2000toInf_TuneCP5_13TeV-madgraph-pythia8': 20.23, 'QCD_HT300to500_TuneCP5_13TeV-madgraphMLM-pythia8': 322600, 'QCD_HT500to700_TuneCP5_13TeV-madgraphMLM-pythia8': 29980, 'QCD_HT700to1000_TuneCP5_13TeV-madgraphMLM-pythia8': 6334, 'QCD_HT1000to1500_TuneCP5_13TeV-madgraphMLM-pythia8': 1088, 'QCD_HT1500to2000_TuneCP5_13TeV-madgraphMLM-pythia8': 99.11, 'QCD_HT2000toInf_TuneCP5_13TeV-madg

In [17]:
# This dictionary tells us how to group the simulated samples together sensibly
# For example, QCD is generated separately for different energies. Combine them
with open('pmap_mc.json') as f:
  pmap = json.load(f)
print(pmap)

{'ZH': ['ZH_HToBB_ZToQQ_M125_13TeV_powheg_pythia8', 'ZH_HToBB_ZToNuNu_M125_13TeV_powheg_pythia8', 'ZH_HToBB_ZToLL_M125_13TeV_powheg_pythia8', 'ggZH_HToBB_ZToQQ_M125_13TeV_powheg_pythia8', 'ggZH_HToBB_ZToNuNu_M125_13TeV_powheg_pythia8', 'ggZH_HToBB_ZToLL_M125_13TeV_powheg_pythia8,'], 'WH': ['WminusH_HToBB_WToLNu_M125_13TeV_powheg_pythia8', 'WminusH_HToBB_WToQQ_M125_13TeV_powheg_pythia8', 'WplusH_HToBB_WToLNu_M125_13TeV_powheg_pythia8', 'WplusH_HToBB_WToQQ_M125_13TeV_powheg_pythia8'], 'ttH': ['ttHTobb_M125_TuneCP5_13TeV-powheg-pythia8'], 'VBF': ['VBFHToBB_M-125_13TeV_powheg_pythia8_weightfix'], 'ggF': ['GluGluHToBB_M-125_13TeV_powheg_MINLO_NNLOPS_pythia8'], 'ggF-powheg': ['GluGluHToBB_M125_TuneCP5_13TeV-powheg-pythia8', 'GluGluHToBB_M125_13TeV_powheg_pythia8'], 'QCD': ['QCD_HT300to500_TuneCP5_13TeV-madgraph-pythia8', 'QCD_HT500to700_TuneCP5_13TeV-madgraph-pythia8', 'QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8', 'QCD_HT1000to1500_TuneCP5_13TeV-madgraph-pythia8', 'QCD_HT1500to2000_TuneC

In [18]:
# I like to run this for one year at a time
year = '2017'
outsum = processor.dict_accumulator()

In [19]:
# Higgs mass window
mbb_min = 110
mbb_max = 138

In [20]:
# Number-counting significance as a function of the number of signal (s) and background (b) events
def significance(s,b):
    if b==0:
        return 0
    z_squared = 2.0*(s+b)*np.log(1.0+1.0*s/b) - 2.0*s
    return np.sqrt(z_squared)

In [21]:
# The quickest option here: copy the pickle file from my area:
# /uscms/home/jennetd/nobackup/hbb-prod-modes/march-2021/vbf-ddbopt/pickles/templates.pkl

# To save time, I run the "load all files" step below once and pickle templates
# Then read in one histogram instead of loading all files each time
# First time I run with repickle=True. Else run with repickle=False
repickle=True

# Check if pickle exists, and don't re-create it if it does
picklename = 'pickles/templates.pkl'
if os.path.isfile(picklename):
    repickle=False

In [22]:
# Load all files - this takes a while
if repickle:
    nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
    for n in range(1,nfiles+1):

        with open('infiles-split/'+year+'_'+str(n)+'.json') as f:
            infiles = json.load(f)
        
        # Skip data - we are dealing with only simulation right now
        if 'JetHT' in infiles.keys():
            continue
        if 'SingleMuon' in infiles.keys():
            continue
    
        filename = '/myeosdir/vbf-ddbopt/outfiles/'+year+'_'+str(n)+'.coffea'
        #filename = 'outfiles/'+year+'_'+str(n)+'.coffea'
        if os.path.isfile(filename):
            out = util.load(filename)
            outsum.add(out)
            print('Loaded file '+str(n))
        else:
            print('Missing file '+str(n),infiles.keys())
            #print("File " + filename + " is missing")
        
    scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
    outsum['templates1'].scale(scale_lumi, 'dataset')
    
    # Use pmap to group the datasets together
    templates = outsum['templates1'].group('dataset', hist.Cat('process', 'Process'), pmap)
    
    # Select out the signal region
    templates = templates.integrate('region', 'signal')

    outfile = open(picklename, 'wb')
    pickle.dump(templates, outfile, protocol=-1)
    outfile.close()

Loaded file 1
Loaded file 2


ValueError: Cannot add this histogram with histogram <Hist (dataset,region,systematic,pt1,msd1,ddb1,deta,mjj) instance at 0x7f7dd2f6a640> of dissimilar dimensions

In [ ]:
# Read the histogram from the pickle file
templates = pickle.load(open(picklename,'rb'))

In [ ]:
# Sum over all bins and print the total number of events per process
templates.sum('msd1','deta','mjj','ddb1').values()

In [ ]:
# Sum the histogram over all but one variable
x = templates.sum('deta','mjj','ddb1')

In [ ]:
# draw the plot of soft drop mass
# NB: the Higgs mass window is 110-138 GeV
plot_mconly_vbf(x,'msd1','msd1')

In [ ]:
# Select out events in the Higgs mass window
templates_window=templates.integrate('msd1',int_range=slice(mbb_min,mbb_max))

In [ ]:
# Apply a cut
templates_cut = templates_window.integrate('deta',int_range=slice(3.5,7)).integrate('mjj',int_range=slice(1000,4000))

# Print the number of events passing the cut per process
templates_cut.sum('ddb1').values()

In [ ]:
sr = templates_cut.sum().values()

# Get the number of signal and background events
s = sr[('VBF',)] 
b = sr[('QCD',)] + sr[('Zjets',)] + sr[('Wjets',)] + sr[('VV',)] + sr[('ttbar',)] +sr[('singlet',)] + sr[('ggF',)] + sr[('WH',)] + sr[('ZH',)] + sr[('ttH',)]

# Print the significance
print(significance(s,b))
# Note: this only makes sense in the mass window of Higgs